In [1]:
import time
from board import SCL, SDA
from busio import I2C
from adafruit_pca9685 import PCA9685
from adafruit_motor import servo
import Jetson.GPIO as GPIO
import ipywidgets.widgets as widgets
from IPython.display import display
import traitlets
from jetbot import Camera, bgr8_to_jpeg
import os 

i2c_bus = I2C(SCL, SDA)
pca = PCA9685(i2c_bus)
pca.frequency = 50

# Create functions and find angles where wheels are parallel (para())

In [13]:
camera_tilt = servo.Servo(pca.channels[6], min_pulse=500, max_pulse=2600, actuation_range=180)
camera_pan = servo.Servo(pca.channels[7], min_pulse=500, max_pulse=2600, actuation_range=180)
camera_z = servo.Servo(pca.channels[8], min_pulse=500, max_pulse=2600, actuation_range=180)

frnt_rw = servo.ContinuousServo(pca.channels[0])
frnt_rs = servo.Servo(pca.channels[1], min_pulse=500, max_pulse=2600, actuation_range=180)

mid_rw = servo.ContinuousServo(pca.channels[2])
mid_rs = servo.Servo(pca.channels[3], min_pulse=500, max_pulse=2600, actuation_range=180)

back_rw = servo.ContinuousServo(pca.channels[4])
back_rs = servo.Servo(pca.channels[5], min_pulse=500, max_pulse=2600, actuation_range=180)

back_lw = servo.ContinuousServo(pca.channels[9])
back_ls = servo.Servo(pca.channels[10], min_pulse=500, max_pulse=2600, actuation_range=180)

mid_lw = servo.ContinuousServo(pca.channels[11])
mid_ls = servo.Servo(pca.channels[12], min_pulse=500, max_pulse=2600, actuation_range=180)

frnt_lw = servo.ContinuousServo(pca.channels[13])
frnt_ls = servo.Servo(pca.channels[14], min_pulse=500, max_pulse=2600, actuation_range=180)

r_wheels = [frnt_rw, mid_rw, back_rw]
l_wheels = [frnt_lw, mid_lw, back_lw]
a_wheels = [frnt_lw, mid_lw, back_lw, frnt_rw, mid_rw, back_rw]

    

a_steer = [frnt_rs, mid_rs, back_rs, frnt_ls, mid_ls, back_ls]

def para():
    frnt_rs.angle = 85
    mid_rs.angle = 100
    back_rs.angle = 110

    frnt_ls.angle = 75
    mid_ls.angle = 90
    back_ls.angle = 85
    time.sleep(.2)                              


In [14]:
camera_tilt.angle = 90
camera_pan.angle=90
camera_z.angle=40

def go_steer_r():
    for wheel in a_wheels:
        wheel.throttle=0.2
        
    time.sleep(.2)
    
    for wheel in a_wheels:
        wheel.throttle=0

def go_steer_l():
    for wheel in a_wheels:
        wheel.throttle=-0.2
        
    time.sleep(.2)
    
    for wheel in a_wheels:
        wheel.throttle=0


def steer_r(change):
    frnt_rs.angle=70
    mid_rs.angle=100
    back_rs.angle=125
    
    frnt_ls.angle=85
    mid_ls.angle=90
    back_ls.angle=70
    time.sleep(.2)
    
    go_steer_r()
    
    
def steer_l(change):
    frnt_rs.angle=70
    mid_rs.angle=100
    back_rs.angle=125
    
    frnt_ls.angle=85
    mid_ls.angle=90
    back_ls.angle=70
    
    time.sleep(.2)
    
    go_steer_l()

def go(change):
    para()
    for wheel in r_wheels:
        wheel.throttle = -1
    for wheel in l_wheels:
        wheel.throttle = 1
    time.sleep(1)
    for wheel in a_wheels:
        wheel.throttle=0
        
def backwardo(change):
    para()
    for wheel in r_wheels:
        wheel.throttle = 1
    for wheel in l_wheels:
        wheel.throttle = -1
    time.sleep(1)
    for wheel in a_wheels:
        wheel.throttle=0   

def tilt_plus(change):
    if camera_tilt.angle <= 170:
        camera_tilt.angle +=20
def tilt_minus(change):
    if camera_tilt.angle >= 10:
        camera_tilt.angle -= 10
def pan_plus(change):
    if camera_pan.angle <= 170:
        camera_pan.angle += 10
def pan_minus(change):
    if camera_pan.angle >= 10:
        camera_pan.angle -= 10
def stop(change):
    camera_pan.angle = 90
    camera_tilt.angle = 90
    camera_z.angle=40
    para()
    
def z_plus(change):
    if camera_z.angle <=130:
        camera_z.angle += 10

def z_minus(change):
    if camera_z.angle >= 50:
        camera_z.angle -= 10
def perp_plus(change):
    frnt_rs.angle=0
    mid_rs.angle=10
    back_rs.angle=20

    frnt_ls.angle=160
    mid_ls.angle=180
    back_ls.angle=175
    time.sleep(.2)
    
    for wheel in r_wheels:
        wheel.throttle = 1
    for wheel in l_wheels:
        wheel.throttle = 1
    time.sleep(1)
    for wheel in a_wheels:
        wheel.throttle=0
        
def perp_min(change):
    frnt_rs.angle=0
    mid_rs.angle=10
    back_rs.angle=20

    frnt_ls.angle=160
    mid_ls.angle=180
    back_ls.angle=175
    time.sleep(.2)
    
    for wheel in r_wheels:
        wheel.throttle = -1
    for wheel in l_wheels:
        wheel.throttle = -1
    time.sleep(1)
    for wheel in a_wheels:
        wheel.throttle=0

# Create control buttons then assign them functions based on (change)

Another way to use traitlets, is by attaching functions (like ``forward``) to events.  These
functions will get called whenever a change to the object occurs, and will be passed some information about that change
like the ``old`` value and the ``new`` value.  

Let's create and display some buttons that we'll use to control the robot.

In [15]:
# create buttons
button_layout = widgets.Layout(width='80px', height='60px', align_self='center')
stop_button = widgets.Button(description='90', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='tilt+', layout=button_layout)
backward_button = widgets.Button(description='tilt-', layout=button_layout)
left_button = widgets.Button(description='pan-', layout=button_layout)
right_button = widgets.Button(description='pan+', layout=button_layout)
another_right = widgets.Button(description='right', button_style='success', layout=button_layout)
another_left = widgets.Button(description='left', button_style='success', layout=button_layout)
another_top = widgets.Button(description='forward', button_style='success', layout=button_layout)
another_bottom = widgets.Button(description='backward', button_style='success', layout=button_layout)
lateral_right = widgets.Button(description='lat+', button_style='success', layout=button_layout)
lateral_left = widgets.Button(description='lat-', button_style='success', layout=button_layout)
z_up = widgets.Button(description='z-up', layout=button_layout)
z_dwn = widgets.Button(description='z-dwn', layout=button_layout)

# display buttons
middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
top_box = widgets.HBox([another_left, forward_button, another_right], layout=widgets.Layout(align_self='center'))
bottom_box = widgets.HBox([lateral_left, backward_button, lateral_right], layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([z_up, top_box, middle_box, bottom_box, z_dwn])
display(controls_box, another_top, another_bottom)

Button(button_style='success', description='forward', layout=Layout(align_self='center', height='60px', width=…

Button(button_style='success', description='backward', layout=Layout(align_self='center', height='60px', width…

You should see a set of robot controls displayed above!  But right now they wont do anything.  To do that
we'll need to create some functions that we'll attach to the button's ``on_click`` event.  

Now that we've defined the functions, let's attach them to the on-click events of each button

In [16]:
# link buttons to actions
stop_button.on_click(stop)
forward_button.on_click(tilt_minus)
backward_button.on_click(tilt_plus)
left_button.on_click(pan_plus)
right_button.on_click(pan_minus)
another_top.on_click(go)
another_bottom.on_click(backwardo)
lateral_right.on_click(perp_plus)
lateral_left.on_click(perp_min)
another_right.on_click(steer_r)
another_left.on_click(steer_l)
z_up.on_click(z_plus)
z_dwn.on_click(z_minus)
#z_up.on_click()


Now when you click each button, you should see the robot move!

# Data collection for training the model

In [6]:
blocked_dir = 'dataset/blocked'
free_dir = 'dataset/free'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(free_dir)
    os.makedirs(blocked_dir)
except FileExistsError:
    print('Directories not created becasue they already exist')

Directories not created becasue they already exist


Camera & button widges

In [17]:
camera = Camera.instance(width=224, height=224)

image = widgets.Image(format='jpeg', width=400, height=400)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

button_layout = widgets.Layout(width='128px', height='64px')
free_button = widgets.Button(description='add free', button_style='success', layout=button_layout)
blocked_button = widgets.Button(description='add blocked', button_style='danger', layout=button_layout)
free_count = widgets.IntText(layout=button_layout, value=len(os.listdir(free_dir)))
blocked_count = widgets.IntText(layout=button_layout, value=len(os.listdir(blocked_dir)))

display(widgets.HBox([free_count, free_button]))
display(widgets.HBox([blocked_count, blocked_button]))
display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [8]:
from uuid import uuid1

def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

def save_free():
    global free_dir, free_count
    save_snapshot(free_dir)
    free_count.value = len(os.listdir(free_dir))
    
def save_blocked():
    global blocked_dir, blocked_count
    save_snapshot(blocked_dir)
    blocked_count.value = len(os.listdir(blocked_dir))
    
# attach the callbacks, we use a 'lambda' function to ignore the
# parameter that the on_click event would provide to our function
# because we don't need it.
free_button.on_click(lambda x: save_free())
blocked_button.on_click(lambda x: save_blocked())

## Next

Once you've collected enough data, we'll need to copy that data to our GPU desktop or cloud machine for training.  First, we can call the following *terminal* command to compress
our dataset folder into a single *zip* file.

> The ! prefix indicates that we want to run the cell as a *shell* (or *terminal*) command.

> The -r flag in the zip command below indicates *recursive* so that we include all nested files, the -q flag indicates *quiet* so that the zip command doesn't print any output

In [11]:
!zip -r -q dataset.zip dataset

You should see a file named ``dataset.zip`` in the Jupyter Lab file browser.  You should download the zip file using the Jupyter Lab file browser by right clicking and selecting ``Download``.

Next, we'll need to upload this data to our GPU desktop or cloud machine (we refer to this as the *host*) to train the collision avoidance neural network.  We'll assume that you've set up your training
machine as described in the JetBot WiKi.  If you have, you can navigate to ``http://<host_ip_address>:8888`` to open up the Jupyter Lab environment running on the host.  The notebook you'll need to open there is called ``collision_avoidance/train_model.ipynb``.

So head on over to your training machine and follow the instructions there!  Once your model is trained, we'll return to the robot Jupyter Lab enivornment to use the model for a live demo!